### data analysis stage:
#### 1. Join the data
#### 2. Research ways to analyse how each of the features affects the probability of a transaction to be fraud

In [33]:
import pandas as pd

df_train = pd.read_csv('dataset/model_dataset/transactions_train.csv')
column_name = 'IS_RECURRING_TRANSACTION'

def clean_column(value):
    if value is True:
        return "True"
    elif (value is False) or (value=="Fals"):
        return "False"
    return str(value)

# Apply the function to the column
df_train[column_name] = df_train[column_name].apply(clean_column)

df_train.to_csv('dataset/model_dataset/transactions_train.csv', index=False)

print(f"Column '{column_name}' cleaned and saved successfully!")

C:\Users\bbdnet2520\AppData\Local\Temp\ipykernel_11336\2353394586.py:3: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('dataset/model_dataset/transactions_train.csv')


Column 'IS_RECURRING_TRANSACTION' cleaned and saved successfully!


In [46]:
import numpy as np

df = pd.read_csv('dataset/model_dataset/full_transactions_train.csv')

df['CUST_TO_TERM_DIST'] = np.sqrt((df['x_terminal_id'] - df['x_customer_id']) ** 2 +
                                  (df['y_terminal__id'] - df['y_customer_id']) ** 2)
df.drop(columns=['x_customer_id', 'y_customer_id', 'x_terminal_id', 'y_terminal__id', 'CUSTOMER_ID', 'TERMINAL_ID'], inplace=True)
df.to_csv('cleaned_file_with_distances.csv', index=False)


C:\Users\bbdnet2520\AppData\Local\Temp\ipykernel_11336\4166422411.py:3: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dataset/model_dataset/full_transactions_train.csv')


In [53]:
import pandas as pd

df = pd.read_csv('cleaned_file_with_distances.csv')
print(df.columns)  # Check if the new column exists in the saved CSV
print(df[['CUST_TO_TERM_DIST']].head())
#full_t_train = pd.read_csv('dataset/model_dataset/full_transactions_train.csv', index_col='TX_ID')
#full_t_test = pd.read_csv('dataset/model_dataset/full_transactions_test.csv', index_col='TX_ID')
#transactions_train = pd.read_csv('dataset/model_dataset/transactions_train.csv')
#transactions_test = pd.read_csv('dataset/model_dataset/transactions_test.csv')
#customers = pd.read_csv('dataset/model_dataset/customers.csv', index_col='CUSTOMER_ID')
#merchants = pd.read_csv('dataset/model_dataset/merchants.csv', index_col='MERCHANT_ID')
#terminals = pd.read_csv('dataset/model_dataset/terminals.csv', index_col='TERMINAL_ID')
##Merge transactions with customers on 'customer_id'
#transactions_with_customers = pd.merge(transactions_train, customers, on='CUSTOMER_ID', how='left')
#transactions_with_customers_terminals = pd.merge(transactions_with_customers, terminals, on='TERMINAL_ID', how='left')
#full_transactions = pd.merge(transactions_with_customers_terminals, merchants, on='MERCHANT_ID', how='left')
#print(full_transactions.head())
#full_transactions.to_csv('dataset/model_dataset/full_transactions_train.csv', index=False)


C:\Users\bbdnet2520\AppData\Local\Temp\ipykernel_11336\2328511768.py:3: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cleaned_file_with_distances.csv')


Index(['TX_ID', 'TX_TS', 'TX_AMOUNT', 'TX_FRAUD',
       'TRANSACTION_GOODS_AND_SERVICES_AMOUNT', 'TRANSACTION_CASHBACK_AMOUNT',
       'CARD_EXPIRY_DATE', 'CARD_DATA', 'CARD_BRAND', 'TRANSACTION_TYPE',
       'TRANSACTION_STATUS', 'FAILURE_CODE', 'FAILURE_REASON',
       'TRANSACTION_CURRENCY', 'CARD_COUNTRY_CODE', 'MERCHANT_ID',
       'IS_RECURRING_TRANSACTION', 'ACQUIRER_ID', 'CARDHOLDER_AUTH_METHOD',
       'BUSINESS_TYPE', 'MCC_CODE', 'LEGAL_NAME', 'FOUNDATION_DATE',
       'TAX_EXCEMPT_INDICATOR', 'OUTLET_TYPE', 'ACTIVE_FROM', 'TRADING_FROM',
       'ANNUAL_TURNOVER_CARD', 'ANNUAL_TURNOVER', 'AVERAGE_TICKET_SALE_AMOUNT',
       'PAYMENT_PERCENTAGE_FACE_TO_FACE', 'PAYMENT_PERCENTAGE_ECOM',
       'PAYMENT_PERCENTAGE_MOTO', 'DEPOSIT_REQUIRED_PERCENTAGE',
       'DEPOSIT_PERCENTAGE', 'DELIVERY_SAME_DAYS_PERCENTAGE',
       'DELIVERY_WEEK_ONE_PERCENTAGE', 'DELIVERY_WEEK_TWO_PERCENTAGE',
       'DELIVERY_OVER_TWO_WEEKS_PERCENTAGE', 'CUST_TO_TERM_DIST'],
      dtype='object')
   CUST_

In [54]:
cols_missing = [col for col in df.columns if df[col].isnull().any()]
print(cols_missing)

['FAILURE_CODE', 'FAILURE_REASON', 'ACQUIRER_ID', 'CARDHOLDER_AUTH_METHOD']


In [55]:
from sklearn.model_selection import train_test_split

# Remove rows with missing target, separate target from predictors
df.dropna(axis=0, subset=['TX_FRAUD'], inplace=True)
y = df.TX_FRAUD
df.drop(['TX_FRAUD'], axis=1, inplace=True)

# To keep things simple, we'll drop columns with missing values
df.drop(cols_missing, axis=1, inplace=True)
print(df.columns)

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(df, y, train_size=0.8, test_size=0.2, random_state=0)

Index(['TX_ID', 'TX_TS', 'TX_AMOUNT', 'TRANSACTION_GOODS_AND_SERVICES_AMOUNT',
       'TRANSACTION_CASHBACK_AMOUNT', 'CARD_EXPIRY_DATE', 'CARD_DATA',
       'CARD_BRAND', 'TRANSACTION_TYPE', 'TRANSACTION_STATUS',
       'TRANSACTION_CURRENCY', 'CARD_COUNTRY_CODE', 'MERCHANT_ID',
       'IS_RECURRING_TRANSACTION', 'BUSINESS_TYPE', 'MCC_CODE', 'LEGAL_NAME',
       'FOUNDATION_DATE', 'TAX_EXCEMPT_INDICATOR', 'OUTLET_TYPE',
       'ACTIVE_FROM', 'TRADING_FROM', 'ANNUAL_TURNOVER_CARD',
       'ANNUAL_TURNOVER', 'AVERAGE_TICKET_SALE_AMOUNT',
       'PAYMENT_PERCENTAGE_FACE_TO_FACE', 'PAYMENT_PERCENTAGE_ECOM',
       'PAYMENT_PERCENTAGE_MOTO', 'DEPOSIT_REQUIRED_PERCENTAGE',
       'DEPOSIT_PERCENTAGE', 'DELIVERY_SAME_DAYS_PERCENTAGE',
       'DELIVERY_WEEK_ONE_PERCENTAGE', 'DELIVERY_WEEK_TWO_PERCENTAGE',
       'DELIVERY_OVER_TWO_WEEKS_PERCENTAGE', 'CUST_TO_TERM_DIST'],
      dtype='object')


In [56]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    print(preds)
    return mean_absolute_error(y_valid, preds)

In [57]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

In [43]:
print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
[0.03 0.   0.14 ... 0.1  0.   0.01]
0.03328527624620232


In [61]:
#print(X_train.select_dtypes(include=['object']).columns)

# ts_tx can't use ordinal/one-hot encoding
# card_expiry_date can't use ordninal/one-hot encoding
# merchant_id can't use ordninal/one-hot encoding

cols = ['CARD_DATA',
       'CARD_BRAND', 'TRANSACTION_TYPE', 'TRANSACTION_STATUS',
       'TRANSACTION_CURRENCY', 'CARD_COUNTRY_CODE',
       'IS_RECURRING_TRANSACTION', 'BUSINESS_TYPE', 'LEGAL_NAME',
       'TAX_EXCEMPT_INDICATOR', 'OUTLET_TYPE',
       'CUST_TO_TERM_DIST']

for col in cols:
  print(X_train[col].unique())
  print(X_valid[col].unique())
  print()

['4929********693' '4916********991' '5551********560' ...
 '3427********767' '3409********256' '3777********251']
['4916********912' '4929********541' '4485********231' ...
 '3409********546' '3409********103' '3427********751']

['Visa' 'MasterCard' 'AMEX' 'Discover']
['Visa' 'AMEX' 'MasterCard' 'Discover']

['Refund' 'Purchase' 'Purchase\xa0with\xa0cashback'
 'Cash\xa0Advance/Withdrawal']
['Purchase' 'Cash\xa0Advance/Withdrawal' 'Purchase\xa0with\xa0cashback'
 'Refund']

['Settled' 'Captured' 'Rejected' 'Authorized']
['Settled' 'Captured' 'Authorized' 'Rejected']

['GBP' 'EUR' 'CHF' 'CNY' 'CAD' 'JPY' 'USD' 'AED' 'RON' 'HKD' 'MDL']
['GBP' 'EUR' 'HKD' 'USD' 'CHF' 'CNY' 'JPY' 'CAD' 'RON' 'AED' 'MDL']

['GB' 'BE' 'US' 'DE' 'CA' 'AE' 'FR' 'NO' 'HR' 'DK' 'RO' 'FI' 'BT' 'SA']
['GB' 'BE' 'US' 'RO' 'NO' 'FR' 'DE' 'HR' 'DK' 'CA' 'SA' 'BT' 'FI' 'AE']

[False True 'False' 'True' 'Fals']
[False True 'False' 'True']

['S Corporations' 'Sole Proprietorships' 'Limited Liability Company (LLC)'
 'Cor

In [40]:
# Categorical columns in the training data
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely ordinal encoded
good_label_cols = [col for col in object_cols if
                   set(X_valid[col]).issubset(set(X_train[col]))]

# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))

print('Categorical columns that will be ordinal encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be ordinal encoded: ['CARD_EXPIRY_DATE', 'CARD_BRAND', 'TRANSACTION_TYPE', 'TRANSACTION_STATUS', 'TRANSACTION_CURRENCY', 'CARD_COUNTRY_CODE']

Categorical columns that will be dropped from the dataset: ['TX_TS', 'TX_ID', 'CARD_DATA']


In [41]:
from sklearn.preprocessing import OrdinalEncoder

# Drop categorical columns that will not be encoded
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# Apply ordinal encoder
ordinal_encoder = OrdinalEncoder()
label_X_train[good_label_cols] = ordinal_encoder.fit_transform(X_train[good_label_cols])
label_X_valid[good_label_cols] = ordinal_encoder.transform(X_valid[good_label_cols])

In [42]:
print("MAE from Approach 2 (Ordinal Encoding):")
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Ordinal Encoding):
[0.03 0.   0.07 ... 0.05 0.01 0.03]
0.03744805146093545
